In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from tqdm import tqdm_notebook as tqdm

## Định nghĩa trước

- total - Tổng số dòng có trong file dữ liệu chưa chuẩn hóa.
- chunksize - Số lượng dữ liệu được sử dụng để tính mỗi dòng trong file đã chuẩn hóa.
- segments - Số lượng lượng dữ liệu trong file đã chuẩn hóa.

In [ ]:
%%time
# Tổng số dòng dữ liệu
total = 629145481 
chunksize = 150_000

c = 1

segments = total // chunksize
# Debug
# segments = 10

In [ ]:
# Hiển thị đến độ chính xác 15 chữ số sau dấu phẩy.
pd.options.display.precision = 15

# Training

#### Kiểm tra phiên bản của sklearn
Code bên dưới để kiểm tra sklearn có sử dụng được các thuật toán từ bản 0.23 trở lên hay không. Một số thuật toán không xuất hiện trong các version từ 0.22 trở xuống, như VotingRegressor.

In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

#### Import các thư viện cần thiết cho quá trình nạp dữ liệu

In [ ]:
%%time
# Thuật toán KMean từ sklearn để phân cụm dữ liệu.
from sklearn.cluster import KMeans

# thư viện các thao tác với mảng số.
import numpy as np

# thư viện trực quan hóa.
import matplotlib.pyplot as plt 
import seaborn as sns

# thư viện quản lý các Warning (thông báo cảnh cáo).
import warnings

# Bỏ qua các warning.
warnings.filterwarnings('ignore')

#### Nạp các đầu vào cho huấn luyện
Sử dụng đầu vào là các file đã được xử lý ở [notebook](https://www.kaggle.com/ngodang/int3405e-20-lanl-linear-boosting), tiến hành nạp các đầu vào bằng pandas.

In [ ]:
X = pd.read_csv('../input/lanl-u-vo-chun-ha/X_train.csv')
y = pd.read_csv('../input/lanl-u-vo-chun-ha/y_train.csv')
len(X.columns)

#### Trực quan một số giá trị 

Trực quan các giá trị của time_to_failure theo thời gian.

In [ ]:
plt.plot(range(len(y)), y)
plt.title("Giá trị của time_to_failure theo thời gian")
plt.show()

In [ ]:
sns.histplot(data=y, x="time_to_failure")
plt.title("Phân bố nhãn y trong tập kiểm thử")
plt.show()

Biểu đồ các giá trị của TTF theo thời gian. Ta thấy rằng do số mẫu rất nhỏ so với tổng số mẫu, nên tính liên tục của y không bị ảnh hưởng quá nhiều (dữ liệu vẫn có cách hình răng cưa).

Biểu đồ phân bố nhãn y trong tập kiểm thử, ta thấy có sự mất cân bằng lớn khi những điểm dữ liệu có nhãn càng lớn thì càng ít dữ liệu.

#### Nạp các đầu vào cho bộ test.

In [ ]:
# Nạp dữ liệu từ file bản nộp mẫu, để thuận tiện cho việc xuất ra file kết quả.
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id')

X_test = pd.read_csv('../input/lanl-u-vo-chun-ha/X_test.csv')

#### Chuẩn hóa dữ liệu

Sử dụng Standard Scaler, dữ liệu sẽ được chuẩn hóa về dạng có mean bằng 0 và std bằng 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_test_scaled = scaler.transform(X_test)

#### Định nghĩa hàm mất mát.

Trong đề bài yêu cầu sử dụng hàm đánh giá là MAE hay Mean Absolute Error, được tính bằng trung bình trị tuyệt đối sự khác nhau giữa dữ liệu nhãn dự đoán và dữ liệu nhãn được cho.

Dữ liệu được tính ở đây là time_to_failure.

Nạp hàm tính MAE từ sklearn:

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

# Training

## Chia dữ liệu thành tập huấn luyện và thẩm định.
Chia bộ dữ liệu để huấn luyện thành 2 tập: Huấn luyện (training) và thẩm định (validation) sử dụng hàm train_test_split từ sklearn.model_selection. 

Tỉ lệ dữ liệu dùng để huấn luyện sẽ là 50%. Do số file test là 2624 file, số dòng trong file training là 4194. Việc chia một nửa sẽ làm số lượng file cho validation gần với số lượng file trong test. 

Dữ liệu được sử dụng là 2 loại:
- \<X>_no_shuffle_scaled: Là dữ liệu có phần valid là liên tục, sử dụng để trực quan khả năng dự đoán của mô hình. Được gọi là tập không tráo.
- Loại còn lại được tráo và chia ngay từ đầu, sử dụng để tìm ra mô hình tốt nhất. Được gọi là tập có tráo.

In [ ]:
train_size = .5

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X_train_no_shuffle_scaled, X_valid_no_shuffle_scaled, y_train_no_shuffle, y_valid_no_shuffle = train_test_split(X_scaled, y.values, train_size=train_size, shuffle=False)
X_train_no_shuffle_scaled, y_train_no_shuffle = shuffle(X_train_no_shuffle_scaled, y_train_no_shuffle, random_state=42)

X_train_scaled, X_valid_scaled, y_train, y_valid = train_test_split(X_scaled, y.values, train_size=train_size, shuffle=True)

## Định nghĩa một hàm trực quan phục vụ thẩm định mô hình.

In [ ]:
def plot_valid(y_valid, y_predict):
    plt.subplots(figsize=(13, 3))
    plt.plot(range(len(y_valid)), y_valid, label="valid")
    plt.plot(range(len(y_predict)), y_predict, label="predict")
    plt.legend()
    plt.show()
    
plot_valid(y_valid_no_shuffle, y_valid_no_shuffle)

## Import các mô hình sẽ sử dụng.

In [ ]:
## Nhóm các mô hình tuyến tính.
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.neural_network import MLPRegressor

## Nhóm mô hình cây.
from sklearn.tree import DecisionTreeRegressor

## Nhóm mô hình Bagging.
from sklearn.ensemble import RandomForestRegressor

## Nhóm mô hình phân cụm.
from sklearn.neighbors import KNeighborsRegressor

## Nhóm mô hình Boosting.
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

## Nhóm mô hinh Ensemble khác.
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

## Nhóm các thuật toán boosting hiệu quả với các cuộc thi của kaggle khác.
import xgboost as xg
import catboost as cb
import lightgbm as lgb

## Phương pháp Cross Validation

Chia tập dữ liệu thành nhiều phần bằng nhau. Ứng với khi xét mỗi phần, thực hiện các bước:
1. Tạo và huấn luyện mô hình trên các phần dữ liệu ngoài phần đang xét.
2. Tính các số liệu về khả năng dự đoán của mô hình trên phần đang xét.
3. Tính trung bình các số liệu.

Mô hình nào có điểm trung bình các số liệu cao hơn sẽ được chọn.

## Cross Validation trong lựa chọn tham số cho mô hình

Hàm sử dụng: GridSearchCV từ sklearn.

Các tham số của lớp:
- estimator: Là mô hình dự đoán đang cần tìm siêu tham số phù hợp
- param_grid: Là một đối tượng từ điển, chứa thông tin về tên siêu tham số và các giá trị có thể được chọn.
- refit: Thực hiện huấn luyện mô hình tốt nhất trên toàn bộ bộ dữ liệu.
- verbose: Quyết định mức độ của việc in ra các log.
- cv: Số phần chia ra cho Cross Validation.
- scoring: metric dùng để đánh giá độ tốt của mô hình.
- n_jobs: Số quy trình có thể chạy song song.

Chức năng: Mỗi tổ hợp các giá trị trong các trường của param_grid sẽ đánh giá độ tốt dùng Cross Validation. Lớp trả về tổ hợp tham số tốt nhất.

## Xây dựng chiến lược đánh giá Out Of Fold (OOF)

Out of Folds là chiến lược sẽ chia bộ dữ liệu ra thành n_folds phần bằng nhau. Ứng với mỗi phần, thuật toán sẽ thực hiện các bước sau:
1. Đặt phần đang xét làm tập valid hiện tại. 
2. Huấn luyện mô hình trên các tập còn lại.
3. Sử dụng mô hình để dự đoán tập valid. Giá trị dự đoán sẽ được ghi vào một mảng $valid$, tại vị trí tương ứng được lấy mẫu làm tập valid. 
5. Nếu vẫn còn tập valid chưa xét, tiếp tục xét và trở về bước 1.
6. Tính hàm đánh giá trên mảng $valid$ để đánh giá so với giá trị nhãn đầu vào.

Có 2 phiên bản cho hàm OOF:
- oof là phiên bản không sử dụng giá trị tốt nhất để đánh giá, hoàn toàn huấn luyện dựa vào dữ liệu training.
- oof_with_evalset là phiên bản sử dụng tập valid để tìm giá đánh giá mô hình.

### oof: 
Hàm dự đoán không sử dụng tập eval set để đánh giá và dừng sớm trong quá trình huấn luyện

In [ ]:
%%time
from sklearn.model_selection import KFold

def oof(params, model_class, X, y, n_folds):
    
    # Lớp KFold để chia dữ liệu thành các Fold.
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    # Mảng Valid để lưu lại các giá trị dự đoán.
    oof_pred = np.zeros((len(y), ))
    
    # Duyệt qua các tập được chia bởi KFold.
    for i, (train_index, valid_index) in enumerate(kfold.split(X)):
        X_train, X_valid = X[train_index], X[valid_index] 
        y_train, y_valid = y[train_index], y[valid_index]
        
        # Xây dựng mô hình.
        model = model_class(**params)
        
        # Hàm lọc bỏ các Warning bắt buộc từ sklearn. 
        # Các hàm Warning của sklearn có khả năng miễn nhiễm với lệnh ignore được khai báo bên trên.
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            
            # Huấn luyện mô hình.
            model.fit(X_train, y_train.ravel())
        
        # Dự đoán cho cho tập X_valid.
        predict_valid = model.predict(X_valid).ravel()
        
        # Gán giá trị cho vị trí lấy mẫu valid tương ứng trong mảng Valid.
        oof_pred[valid_index] = predict_valid
        
    print("Avg MAE: ", MAE(oof_pred, y))

### oof_with_evalset: 
Hàm dự đoán sử dụng tập eval set để đánh giá và dừng sớm trong quá trình huấn luyện. Sử dụng đối với các mô hình cho phép early stopping (Dừng sớm nếu kết quả không cải thiện).

In [ ]:
%%time
from sklearn.model_selection import KFold

def oof_with_evalset(params, model_class, X, y, n_folds):
    
    # Lớp KFold để chia dữ liệu thành các Fold.
    kfold = KFold(n_splits = n_folds, shuffle=True, random_state=42)
    # Mảng Valid để lưu lại các giá trị dự đoán.
    oof_pred = np.zeros((len(y), ))
    
    # Duyệt qua các tập được chia bởi KFold.
    for i, (train_index, valid_index) in enumerate(kfold.split(X)):
        X_train, X_valid = X[train_index], X[valid_index] 
        y_train, y_valid = y[train_index], y[valid_index]
        
        # Xây dựng mô hình.
        model = model_class(**params)
        
        # Hàm lọc bỏ các Warning bắt buộc từ sklearn. 
        # Các hàm Warning của sklearn có khả năng miễn nhiễm với lệnh ignore được khai báo bên trên.
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            
            # Huấn luyện mô hình.
            model.fit(X_train, y_train.ravel(), eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=False)
        
        # Dự đoán cho cho tập X_valid.
        predict_valid = model.predict(X_valid).ravel()
        
        # Gán giá trị cho vị trí lấy mẫu valid tương ứng trong mảng Valid.
        oof_pred[valid_index] = predict_valid
        
    print("Avg MAE: ", MAE(oof_pred, y))

Kiểm tra sử dụng 2 mã bên dưới:

In [ ]:
%%time
params = {}

oof(
    params, LinearRegression, 
    X_train_no_shuffle_scaled, 
    y_train_no_shuffle, 
    5
)

In [ ]:
%%time
params = {'colsample_bytree': 0.4, 'learning_rate': 0.01, 'max_depth': 12, 'n_estimators': 100, 'num_leaves': 127, 'objective': 'regression_l1', 'reg_lambda': 3, 'subsample': 0.4}

oof_with_evalset(
    params, lgb.LGBMRegressor,
    X_train_no_shuffle_scaled, 
    y_train_no_shuffle, 
    5
)

### Hàm trực quan đánh giá

Chạy dữ liệu trên tập không tráo và trực quan kết quả

In [ ]:
def visualize(params, model_class):
    model = model_class(**params)
    model.fit(X_train_no_shuffle_scaled, y_train_no_shuffle)
    
    pred = model.predict(X_valid_no_shuffle_scaled)
    plot_valid(y_valid_no_shuffle, pred)

### Chiến lược tìm kiếm mô hình tốt nhất

Chiến lượng sử dụng là sử dụng Grid Search để tìm ra tổ hợp các tham số hợp lệ đạt điểm Cross Validation cao nhất.

In [ ]:
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)

def training(param_grid, model_class):
    grid = GridSearchCV(model_class(), param_grid, verbose=1, cv=3, scoring='neg_mean_absolute_error', n_jobs=4)
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        
        grid.fit(X_scaled, y.values.ravel()) 

    print(grid.best_estimator_) 
    print(-grid.best_score_)
    return grid

### Chiến lược đánh giá và xuất ra file csv

In [ ]:
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)

def eval_and_export(params, model_class, model_name):

    model = model_class(**params)
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        model.fit(X_scaled, y.values.ravel())

    pred_test = model.predict(X_test_scaled)
    
    # Do số time_to_failure luôn dương, nên báo cáo lấy max với 0
    submission['time_to_failure'] = np.maximum(pred_test, 0)
    submission.to_csv('submission_{}.csv'.format(model_name))
    
    print("Export to submission_{}.csv complete!".format(model_name))

## Linear Regressor

### Mô tả

Hồi quy tuyến tính là phương pháp mô hình hóa quan hệ giữa các thuộc tính bằng cách tìm ra một phương trình đường thẳng trên tập dữ liêu, với tham số là: $W, b$, trong đó, $W$ là một vector cùng số chiều với dữ liệu, và $b$ là bias, là một giá trị vô hướng.

Phườn trình đường thẳng mà hàm hồi quy tuyến tính tìm kiếm có dạng: $Y = W^T X + b$ trong đó, $X$ là các đặc trưng đầu vào, $Y$ là giá trị dữ đoán. Nhiệm vụ của Hồi quy tuyến tính là tìm xác định $W$ và $b$ sao cho k

Hàm sẽ tập trung tối thiểu hóa lỗi $L$, được định nghĩa là mức độ sai số dự đoán của mô hình. Phương pháp xuống đồi bằng đạo hàm và lan truyền ngược có thể được sử dụng để tối thiểu hóa lỗi $L$.

Hàm hồi quy tuyến tính của thư viện sklearn không có tham số liên quan đến huấn luyện.

In [ ]:
%%time
from sklearn.linear_model import LinearRegression

params = {}

In [ ]:
%%time 

oof(params, LinearRegression, X_scaled, y.values, 3)
visualize(params, LinearRegression)
eval_and_export(params, LinearRegression, "LinearRegression")

## Ridge Regressor - L2 Norm


### Mô tả:

Ridge.

Thuật toán được định nghĩa và huấn luyện giống với Hồi quy tuyến tính. Điểm khác nhau duy nhất là trong hàm $L$ của Ridge cộng thêm thành phần $\alpha ||W||^2_2$ (Chuẩn L2). Thành phần này còn gọi là Regularization.

Việc thêm regularization có thể giúp giảm các giá trị của một số hệ số, thậm chí làm biến mất (đưa về $0$), từ đó, chọn các đặc trưng một cách tự động, trong vài trường hợp có thể tránh được overfitting.

Trong lớp Ridge của sklearn, các tham số là:
- alpha: Quyết định ảnh hưởng của Regularization là nhiều hay ít.

Việc sử dụng chuẩn L2 là sẽ tốt hơn nếu các outlier là quan trọng, vì sử dụng L2 sẽ tăng tỉ trọng của các outlier.

In [ ]:
%%time
from sklearn.linear_model import Ridge

parameters = {'alpha': [0.01, 0.1, 0.3, 0.6, 1, 2]}

grid = training(parameters, Ridge)

Siêu tham số dự đoán được là giống nhau.

In [ ]:
%%time
from sklearn.linear_model import Ridge

params = grid.best_params_
oof(params, Ridge, X_scaled, y.values, 3)
visualize(params, Ridge)
eval_and_export(params, Ridge, "Ridge")

## Lasso Regressor - L1 Norm

### Mô tả:

Lasso là viết tắt của Least Absolute Shrinkage and Selection Operator. 

Thuật toán được định nghĩa và huấn luyện giống với Hồi quy tuyến tính. Điểm khác nhau duy nhất là trong hàm $L$ của Lasso cộng thêm thành phần $\alpha ||W||$ (Chuẩn L1). Thành phần này còn gọi là Regularization.

Việc thêm regularization có thể giúp giảm các giá trị của một số hệ số, thậm chí làm biến mất (đưa về $0$), từ đó, chọn các đặc trưng một cách tự động, trong vài trường hợp có thể tránh được overfitting.

Trong lớp Lasso của sklearn, các tham số là:
- alpha: Quyết định ảnh hưởng của Regularization là nhiều hay ít.

Việc sử dụng chuẩn L1 là sẽ tốt hơn nếu muốn khử các outlier. Do L1 sẽ làm giảm giá trị hệ số tương ứng với đặc trưng outlier đến chính xác $0$.

In [ ]:
%%time
from sklearn.linear_model import Lasso

parameters = {'alpha': [0.01, 0.1, 0.3, 0.6, 1, 2]}

grid = training(parameters, Lasso)

In [ ]:
%%time
from sklearn.linear_model import Lasso

params = grid.best_params_
oof(params, Lasso, X_scaled, y.values, 3)
visualize(params, Lasso)
eval_and_export(params, Lasso, "Lasso")

## Elastic Net

Là sự kết hợp của Lasso và Ridge. Công thức của Elastic net tương tự như công thức của Hồi quy tuyến tính, nhưng được cộng thêm thành phần $\alpha * l1\_ratio * ||W|| + 0.5 * \alpha * (1 - l1\_ratio) * ||W||^2_2$.

Trong lớp ElasticNet của sklearn, các tham số là:
- alpha: Quyết định ảnh hưởng của Regularization là nhiều hay ít.
- l1_ratio: Quyết định tỉ lệ bao nhiêu phần của alpha sẽ được dùng làm trọng số cho chuẩn L1, phần còn lại được dùng cho chuẩn L2.

Việc sử dụng chuẩn L1 là sẽ tốt hơn nếu muốn khử các outlier. Do L1 sẽ làm giảm giá trị hệ số đến chính xác $0$.

In [ ]:
%%time
from sklearn.linear_model import ElasticNet

parameters = {
    'alpha': [0.01, 0.1, 0.3, 0.6, 1, 2],
    'l1_ratio': [0.3, 0.5, 0.7]
}

grid = training(parameters, ElasticNet)

In [ ]:
%%time

params = grid.best_params_
oof(params, ElasticNet, X_scaled, y.values, 3)
visualize(params, ElasticNet)
eval_and_export(params, ElasticNet, "ElasticNet")

## SVM

SVM (Support vector machine) gồm 2 loại:
#### SVM lề cứng
là thuật toán hướng đến tìm ra một siêu phẳng sao cho mọi điểm dữ liệu đều được phân loại đúng, và khoảng cách từ điểm dữ liệu được phân loại đúng gần nhất với siêu phẳng phải là lớn nhất, khoảng cách này gọi là lề, được tính bởi công thức: 

$$\delta = \min_{i=1}^n \frac{|\mathbf w^T\mathbf x_i+b|}{\|\mathbf w\|}$$

Tuy nhiên thuật toán yêu cầu là dữ liệu phải khả tách tuyến tính, tức là phải tồn tại một siêu phẳng phân cách dữ liệu. Hàm mục tiêu được tính bằng công thức:

$$\begin{align*}\max_{\mathbf w, b,\delta}&~~\delta\\
\textrm{ràng buộc}&~~ y_i(\mathbf w^T\mathbf x_i+b)\geq \delta\|\mathbf w\|>0,\forall i\end{align*}$$

#### SVM lề mềm: 
Có thể chạy trong trường hợp dữ liệu không khả tách tuyến tính bằng cách sử dụng các slack variable ($\xi_i$) biểu thị cho lượng vi phạm lề của mỗi điểm. Cụ thể, hàm mục tiêu sẽ được tính bởi công thức:

$$\begin{align*}
\min_{\mathbf w, b,\xi}&~~\frac 1 2 \|\mathbf w\|^2+C\sum_{i=1}^n \xi_i\\
\textrm{ràng buộc}&~~ y_i(\mathbf w^T\mathbf x_i+b)\geq 1-\xi_i,\forall i\\
&~~ \xi_i \geq 0,\forall i
\end{align*}$$

Slack variable được tính bằng công thức: 

$$\xi_i = \max(0, 1-y_i(\mathbf w^T\mathbf x_i+b))$$

Lấy $1-y_i(\mathbf w^T\mathbf x_i+b)$ do đây là khoảng cách tới lề ở phía đúng của dữ liệu. Nếu $y_i(\mathbf w^T\mathbf x_i+b)$ trong khoảng $(0, 1]$ thì chứng tỏ dữ liệu phân đúng nhưng vi phạm lề. Nếu $y_i(\mathbf w^T\mathbf x_i+b)$ trong khoảng $(-\inf, 0]$ thì chứng tỏ dữ liệu loại sai. Cả 2 trường hợp đều sẽ làm biến slack lớn hơn 0.

Việc cho biến slack vào hàm mục tiêu sẽ giúp kiểm soát mức độ vi phạm của các điểm dữ liệu, song song với tối đa hóa lề.

### SVR

Support vector regression: Tương tự như SVM, nhưng lần này thuật toán sẽ tìm siêu phẳng và lề sao cho các điểm dữ liệu nằm giữa 2 lề thỏa mãn ràng buộc:
$$\forall n: |y_n-(b+W^T x_n')| \le \epsilon$$

Các tham số chính:
- C: Trọng số cho biến slack
- kernel: Phương pháp ánh xạ đầu vào để có chiều không gian có thể phân lớp tốt hơn.
- epsilon: Khoảng cách tối thiểu mà sẽ không có điểm dữ liệu nào bị đánh trọng số nếu ở trong khoảng đó.

In [ ]:
%%time
from sklearn.svm import SVR
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)

parameters = {
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': [0],
    'gamma': ['scale', 'auto'],
    'C': [0.3, 1],
    'epsilon': [0.05, 0.1, 1]
}

grid = training(parameters, SVR)

In [ ]:
%%time

params = grid.best_params_
oof(params, SVR, X_scaled, y.values, 3)
visualize(params, SVR)
eval_and_export(params, SVR, "SVR")

### NuSVM
Tương tự với SVR, nhưng sử dụng thêm một biến $\nu$ (thay cho $C$) để giám sát số lượng vector mẫu và lỗi của lề (slack variable). $\nu$ là cận trên cho phần lỗi của lề và là cận dưới của phần vector hỗ trợ.

Các tham số chính:
- nu: Tỉ lệ số support vector mong muốn được giữ lại trên tổng số mẫu.
- C: Trọng số cho biến slack
- kernel: Phương pháp ánh xạ đầu vào để có chiều không gian có thể phân lớp tốt hơn.

In [ ]:
%%time
from sklearn.svm import NuSVR

parameters = {
    'nu': [0.3, 0.5, 0.75],
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': [0, 1],
    'gamma': ['scale', 'auto'],
    'C': [0.1, 1, 2]
}

grid = training(parameters, NuSVR)

In [ ]:
%%time

params = grid.best_params_
oof(params, NuSVR, X_scaled, y.values, 3)
visualize(params, NuSVR)
eval_and_export(params, NuSVR, "NuSVR")

## Neural Network

Mạng thần kinh lan truyền (Neural Network - NN). Là kết quả của việc kết hợp nhiều lớp Perceptron lại. Do mỗi lớp perceptron chỉ dự đoán được bằng siêu phẳng, nên việc kết hợp nhiều lớp Perceptron lại sẽ cho phép biểu diện được tính chất phi tuyến.

NN sẽ gồm nhiều lớp perceptron nối tiếp nhau. Mỗi lớp sẽ thực hiện ánh xạ dữ liệu từ số chiều đầu và thành một số chiều đầu ra khác nhau, bằng cách nhận ma trận với ma trận $W_i$ là ma trận biến đổi ở lớp thứ i. 

Ở các lớp khác với lớp cuối, theo sau đầu ra sẽ là một hàm kích hoạt (activation layer). Activation layer có nhiều loại và chức năng khác nhau: 
- identity: không làm gì cả
- logistic: biến đổi bằng công thức $f(x) = frac 1 (1 + \exp(-x))$. Hàm sẽ đảm bảo dữ liệu liên tục và nằm trong khoảng $[0, 1]$
- tanh: biến đổi bằng hàm $f(x) = \tanh(x) = \frac {e^x - e^{-x}} {e^x + e^{-x}}$. Hàm sẽ đảm bảo dữ liệu liên tục và nằm trong khoảng $[-1, 1]$
- relu: triệt tiêu dữ liệu âm, sử dụng hàm $f(x) = \max(0, x)$

Các tham số chính của NN trong sklearn:
- hidden_layer_sizes: là tập số chiều cho các lớp ẩn.
- activation: chọn hàm kích hoạt.
- solver: Phương pháp tối ưu, tiêu biểu có sgd và adam, trong đó sgd sẽ chạy từng giá trị đầu vào một. còn adam có thể chạy với một tập nhiều đầu vào.
- alpha: trọng số L2 cho tham số, tương tự Ridge.
- batch\_size: số lượng dữ liệu trong 1 tập có thể được dùng để huấn luyện cùng nhau.
- learning\_rate: phương pháp được sử dụng để cập nhật tốc độ học trong suốt quá trình huấn luyện. Các phương pháp có thể được sử dụng:
    - constant: Giữ nguyên tốc độ học, không thay đổi trong suốt quá trình huấn luyện.
    - invscaling: Giảm tốc độ học theo từng thời điểm t, bằng cách nhân với một hàm mũ $pow(t, \text{cơ số})$
    - adaptive: Sử dụng chiến lược giảm tốc độ học mỗi khi 2 epoch liên kề không làm giảm giá trị của hàm mất mát.
- learning\_rate\_init: tốc độ học, hệ số được sử dụng khi cập nhật đạo hàm.
- max\_iter: có thể coi như epoch, số lần huấn luyện lại trên tập dữ liệu

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.neural_network import MLPRegressor

parameters = {
    'hidden_layer_sizes': [128, (128, 64)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.01, 0.1],
    'batch_size': [32, 64],
    'learning_rate_init': [0.001, 0.01],
    'learning_rate': ['adaptive'],
    'shuffle': [True],
    'max_iter': [20, 25],
    'early_stopping': [True]
}

grid = training(parameters, MLPRegressor)

In [ ]:
%%time
params = grid.best_params_
oof(params, MLPRegressor, X_scaled, y.values, 3)
visualize(params, MLPRegressor)
eval_and_export(params, MLPRegressor, "MLPRegressor")

## Decision Tree
Cây quyết định (Decision Tree) là một thuật toán xây dựng một cây, hoạt động bằng cách chia bộ dữ liệu thành những phần nhỏ hơn và những phần nhỏ hơn lại tiếp tục được xây dựng tương tự. Cụ thể, xuất phát từ một node gốc, cây sẽ chọn 1 điều kiện và chia bộ dữ liệu tương ứng ở mỗi node thành 2 bộ nhỏ hơn tương ứng với 2 node con. Quá trình tiếp tục nếu gặp điều kiện dừng:
- Đạt độ sâu tối đa
- Số dữ liệu quá ít

Thuật toán xây dựng cây được sklearn sử dụng là CART.

Các tham số chính của cây quyết định:
- criterion: là hàm đánhn giá chất lượng của một lần chia. Do yêu cầu đề bài sử dụng là MAE nên criterion được xét cũng là MAE.
- splitter: Phương án để chia cây tại mỗi node. 
- max_depth: Độ sâu tối thiểu của cây. Nếu đạt được độ sâu này thì cây sẽ không chia nữa.
- min\_samples\_leaf: Số dữ liệu tối thiểu ở lá. Nếu số dữ liệu thấp hơn min_samples_leaf thì cây sẽ không chia nữa.
- max_features: Số lượng cột đặc trưng sẽ được sử dụng.

In [ ]:
%%time
from sklearn.tree import DecisionTreeRegressor

parameters = {
    'criterion': ['mae'],
    'splitter': ['best', 'random'],
    'max_depth': [7, 9, 13],
    'max_features': ['sqrt'],
    'max_leaf_nodes': [8, 16, 32]
}

grid = training(parameters, DecisionTreeRegressor)

In [ ]:
%%time
params = grid.best_params_
oof(params, DecisionTreeRegressor, X_scaled, y.values, 3)
visualize(params, DecisionTreeRegressor)
eval_and_export(params, DecisionTreeRegressor, "DecisionTree")

## Random Forest

Rừng ngẫu nhiên (Random Forest), là thuật toán ensemble, nhưng để ở ngay sau DecisionTree, do rừng thì phải gần cây. Thuật toán sử dụng là tạo một hàm dự đoán dựa trên việc xây dựng các cây quyết định trên các tập dữ liệu con và đưa ra kết quả bằng cách lấy trung bình các dự đoán.

Các tham số chính của random forest trong sklearn:
- n_estimators:  Số lượng cây được xây dựng.
- criterion: Phương pháp đánh giá độ tốt của một cách chia. Yêu cầu đề bài là sử dụng MAE để đánh giá, nên criterion ở đây cũng được đặt là MAE.
- max\_depth: Độ sâu cây tối đa. Nếu một node đạt được độ sâu này sẽ không chia nữa.
- min\_samples\_split: Số lượng mẫu tối thiểu cần để một node có thể chia tiếp. Nếu số lượng mẫu mà nhỏ hơn min_samples_split thì sẽ không chia nữa.
- min\_samples\_leaf: Số lượng mẫu tối thiểu cho một node lá.
- max\_features: Số lượng cột tối đa sẽ được lấy để xây dựng cây.

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

parameters = {
    'n_estimators': [50, 100],
    'criterion': ['mae'],
    'max_depth': [3, 6],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 8]
}

grid = training(parameters, RandomForestRegressor)

In [ ]:
%%time
params = grid.best_params_
oof(params, RandomForestRegressor, X_scaled, y.values, 3)
visualize(params, RandomForestRegressor)
eval_and_export(params, RandomForestRegressor, "RandomForest")

## KNN

K-láng giềng gần nhất hồi quy (K-Nearest Neighbors Regressor), là lớp thuật toán dựa trên phân cụm K láng giềng gần nhất. Mục tiêu được dự đoán bằng cách nội suy cục bộ (local interpolation) trong tập các láng giềng gần nhất trong tập huấn luyện.

Cách nội suy phổ biến nhất là tính trung bình các nhãn của láng giềng. 

Thuật toán để tìm KNN được sử dụng gồm có:
- brute force: Tìm bằng thuật toán thường. Mỗi lần chạy, duyệt qua dữ liệu trong tập train, rồi tính distance, chọn K láng giềng gần nhất. Tốn rất nhiều thời gian.
- K-D Tree: Phương pháp xây dựng cây. Dựa trên việc mã hóa các khoảng cách. Ví dụ, nếu A rất xa so với B, và B lại gần C, thì có thể suy ra C rất xa A. KD Tree có cấu trúc là một cây nhị phân, trong đó, mỗi nhánh sẽ sẽ đại diện cho một vùng quan sát. Do cấu trúc cây nên các vùng quan sát sẽ lồng vào nhau. Cây được xây dựng rất nhanh do không dựa vào các thông tin trong chiều đặc trưng (các cột).
- Ball tree: Phương pháp xây dụng cây mà mỗi node sẽ là một tập các khối siêu cầu (hyper-sphere). Ball tree sẽ chia dữ liệu bằng cách định nghĩa một điểm tâm C và bán kính r, đại diện cho các điểm nằm trong siêu cầu (C, r). Khi tìm kiếm, có thể sử dụng bất đẳng thức tam giác để tìm nhanh hơn.

Các tham số chính trong sklearn:
- n\_neighbors: Số lượng láng giềng sẽ được lấy kết quả.
- weights: Các đánh trọng số ảnh hưởng của các điểm láng giềng đến kết quả hiện tại của điểm đang xét. Có 2 cách đánh trọng số chính là sử dụng trọng số đều (mọi điểm đóng góp như nhau) và trọng số theo khoảng cách (điểm nào gần đốn góp nhiều hơn)
- algorithm: Thuật toán được sử dụng.
- p: Mũ norm của khoảng cách. p = 1, là norm 1, khoảng cách Manhattan. p = 2, là norm 2, khoảng cách Euclide.

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

parameters = {
    'n_neighbors': [3, 7, 13],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree']
}

grid = training(parameters, KNeighborsRegressor)

In [ ]:
%%time

params = grid.best_params_
oof(params, KNeighborsRegressor, X_scaled, y.values, 3)
visualize(params, KNeighborsRegressor)
eval_and_export(params, KNeighborsRegressor, "KNeighbors")

# Họ Ensemble

Là tập hợp các thuật toán sử dụng kết hợp kết quả của các mô hình nhỏ hơn nhằm tổng hợp và đưa ra dự đoán.

## Boosting

Là phương pháp kết hợp các thuật toán, mô hình học được xây dựng dựa trên mô hình trước đó. Phương pháp chủ yếu là đánh trọng số cho các mô hình mới thêm vào, từ đó hạn chế sai lầm của các mô hình trưóc.

Đối với boosting, bài toán tối ưu được đặt ra là:
$$\min _{c_n=1:N, w_n=1:N} L(y, \sum_{n=1}^Nc_nw_n)$$

Trong đó, L là giá trị hàm mất mát, y là nhãn, $w_n$ là mô hình học thứ n, $c_n$ là trọng số của $w_n$.

Không chỉ tìm các mô hình học tốt, Boosting hướng tới xây dựng mô hình học từ các mô hình trước đó, thông qua các trọng số $c_n$. Cụ thể bài toán tối ưu trên có thể được viết thành:
$$\min _{c_n, w_n} L(y, W_{n-1} + c_n w_n)$$

Với $W_{n-1} = \sum _{n=1}^{N-1} c_n w_n$

## CatBoost

Là mô hình hoạt động chính trên các dữ liệu dàng categorical (dữ liệu theo loại). Một ưu điểm nữa là CatBoost có bộ siêu tham số đầu vào khá tốt. Tốn ít thời gian tinh chỉnh tham số.

In [ ]:
%%time

# Tốn nhiều thời gian

import catboost as cb

parameters = {
    'iterations': [30, 50, 100],
    'depth': [6,8,10],
    'learning_rate' : [0.01, 0.05, 0.1],
    'verbose': [False]
}

grid = training(parameters, cb.CatBoostRegressor)
print(grid.best_params_)

In [ ]:
%%time
params = grid.best_params_

oof_with_evalset(params, cb.CatBoostRegressor, X_scaled, y.values, 3)

visualize(params, cb.CatBoostRegressor)

eval_and_export(params, cb.CatBoostRegressor, "catboost")

## Light Gradient Boosting model.

Light Gradient Boosting Model (LightGBM) nối tiếng nhờ nó nhanh và hiệu quả.

Cách hoạt động của LightGBM:
- Sử dụng leaf-wise growth. Xây dựng cây theo từng node, thay vì từng tầng.
- Sau khi thực hiện chia cây, lần chia cây tiếp theo chỉ được thực hiện trên node có giá trị hàm mất mát lớn hơn.
- LGBM sử dụng phương pháp đánh giá dựa vào histogram. Đối với các trường liên tục, thay vì sử dụg các biến độc lập, các giá trị sẽ được chia thành các nhóm liên tục, gọi là các bins. 

Các tham số của LightGBM:
- learning\_rate: Tốc độ học.
- max\_depth: Độ sâu tối đa của cây
- n\_estimators: Số lần thực hiện Boosting
- num\_leaves: Số lượng lá tối đa trong một cây.
- subsample: Tỉ lệ dữ liệu được sử dụng.
- colsample\_bytree: Tỉ lệ số cột được sử dụng để xây dựng một cây.
- colsample\_bynode: Tỉ lệ số cột được sử dụng để chia tại một node.
- lambda\_l1: Là chuẩn L1 của Regularization.
- lambda\_l2: Là chuẩn L2 của Regularization.
- min\_gain\_to\_split: Lượng giá trị giảm của hàm mất mát tối thiểu để thực hiện một phép chia cây.

In [ ]:
%%time
import lightgbm as lgb

parameters = {
    'learning_rate': [0.01],
    'max_depth': [13, 17],
    'n_estimators': [1000],
    'num_leaves': [100],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.8, 1],
    'colsample_bynode': [0.8, 1],
    'reg_lambda': [1, 3],
    'objective': ['regression_l1']
}

grid = training(parameters, lgb.LGBMRegressor)

In [ ]:
params = grid.best_params_

oof_with_evalset(params, lgb.LGBMRegressor, X_scaled, y.values, 3)

visualize(params, lgb.LGBMRegressor)

eval_and_export(params, lgb.LGBMRegressor, "LGBM")

## Voting

Kết hợp nhiều mô hình dự đoán khác nhau và trả về trung bình các giá trị dự đoán.

In [ ]:
%%time
from sklearn.ensemble import VotingRegressor

voting = [
    ('rf', RandomForestRegressor(criterion='mae', max_depth=6, max_features='sqrt')),
    ('nuSVM', NuSVR(C=1, degree=0, nu=0.75)),
    ('svm', SVR(C=1, degree=0, epsilon=1)),
    ('nn', MLPRegressor(activation='logistic', alpha=0.01, batch_size=64,early_stopping=True, hidden_layer_sizes=128,learning_rate='adaptive', learning_rate_init=0.01, max_iter=25)),
    ('knn', KNeighborsRegressor(algorithm='ball_tree', n_neighbors=17, weights='distance'))
]

params = {
    'estimators': voting
}

oof(params, VotingRegressor, X_scaled, y.values, 3)
visualize(params, VotingRegressor)
eval_and_export(params, VotingRegressor, "Voting")

## Stacking

Kết hợp kết quả của nhiều mô hình khác nhau để thực hiện dự đoán. Khác với voting, đầu ra của các mô hình khác nhau sẽ được đánh trọng số bởi một hàm dự đoán cuối.

Các tham số của StackingRegressor của sklearn:
- estimators: Là tập các mô hình dự đoán sẽ được xếp với nhau.
- final\_estimator: Là mô hình được sử dụng để kết hợp tập các mô hình dự đoán với nhau.
- cv: Số lượng tập dữ liệu được chia ra bởi cross-validation.
- n\_jobs: Quy định số lượng quá trình có thể chạy song song.

In [ ]:
%%time
from sklearn.ensemble import StackingRegressor

stacking = [
    ('rf', RandomForestRegressor(criterion='mae', max_depth=6, max_features='sqrt')),
    ('nuSVM', NuSVR(C=1, degree=0, nu=0.75)),
    ('svm', SVR(C=1, degree=0, epsilon=1)),
    ('nn', MLPRegressor(activation='logistic', alpha=0.01, batch_size=64,
             early_stopping=True, hidden_layer_sizes=128,
             learning_rate='adaptive', learning_rate_init=0.01, max_iter=25)),
    ('knn', KNeighborsRegressor(algorithm='ball_tree', n_neighbors=17, weights='distance'))
]

params = {
    'estimators': stacking,
    'cv': 3,
    'n_jobs': 3
}

oof(params, StackingRegressor, X_scaled, y.values, 3)
visualize(params, StackingRegressor)
eval_and_export(params, StackingRegressor, "StackingRegressor")

# Kết quả 